In [1]:
import tensorflow as tf
print(len(tf.config.list_physical_devices('GPU')))
import torch
print(torch.cuda.is_available())

1
True


In [2]:

import pyarabic.araby as ar

# import Stemmer
import functools, operator

import logging

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

## Load Dataset

In [71]:
import pandas as pd
df = pd.read_csv("train.csv")
seed=0

## Arabic stop words

In [72]:
arabic_stop_words=[]
with open ('list.txt',encoding='utf-8') as f :
  for i in f.readlines() :
    arabic_stop_words.append(i)
    arabic_stop_words[-1]=arabic_stop_words[-1][:-1]


In [5]:
!pip install farasapy

In [73]:
import numpy as np
import pandas as pd
import re

#============= Read CSV and apply data preperation =============#


def data_preprocessing (data_frame):
  # clean-up: remove #tags, http links and special symbols
  data_frame['tweet']= data_frame['tweet'].apply(lambda x: x[2:-2])
  data_frame['tweet']= data_frame['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))
  data_frame['tweet'] = data_frame['tweet'].apply(lambda x: re.sub(r'[@|#]\S*', '', x))
  data_frame['tweet'] = data_frame['tweet'].apply(lambda x: re.sub(r'"+', '', x))

  # Remove arabic signs
  data_frame['tweet'] = data_frame['tweet'].apply(lambda x: re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', x))

  # Remove repeated letters like "الللللللللللللللله" to "الله"
  data_frame['tweet'] = data_frame['tweet'].apply(lambda x: x[0:2] + ''.join([x[i] for i in range(2, len(x)) if x[i]!=x[i-1] or x[i]!=x[i-2]]))

  # remove stop words
  data_frame['tweet'] = data_frame['tweet'].apply(lambda x: '' if x in arabic_stop_words else x)

  from nltk.stem.isri import ISRIStemmer
  df['tweet']=df['tweet'].apply(lambda x:ISRIStemmer().stem(x))

  return data_frame


In [74]:
# st =  Stemmer.Stemmer('arabic')
import string,emoji
def data_cleaning (text):
  text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r'^http?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
  text = re.sub(r"http\S+", "", text)
  text = re.sub(r"https\S+", "", text)
  text = re.sub(r'\s+', ' ', text)
  text = re.sub("(\s\d+)","",text)
  text = re.sub(r"$\d+\W+|\b\d+\b|\W+\d+$", "", text)
  text = re.sub("\d+", " ", text)
  text = ar.strip_tashkeel(text)
  text = ar.strip_tatweel(text)
  text = text.replace("#", " ");
  text = text.replace("@", " ");
  text = text.replace("_", " ");
  translator = str.maketrans('', '', string.punctuation)
  text = text.translate(translator)
  em = text
  em_split_emoji = emoji.get_emoji_regexp().split(em)
  em_split_whitespace = [substr.split() for substr in em_split_emoji]
  em_split = functools.reduce(operator.concat, em_split_whitespace)
  text = " ".join(em_split)
  text = re.sub(r'(.)\1+', r'\1', text)
  # text_stem = " ".join([st.stemWord(i) for i in text.split()])
  # text = text +" "+ text_stem
  text = text.replace("آ", "ا")
  text = text.replace("إ", "ا")
  text = text.replace("أ", "ا")
  text = text.replace("ؤ", "و")
  text = text.replace("ئ", "ي")

  return text

In [8]:
df=data_preprocessing(df)
df['tweet']=df['tweet'].apply(lambda x: data_cleaning(x))
df

C:\Users\Kirolos Atef\AppData\Local\Temp\ipykernel_25240\3188261332.py:20: DeprecationWarning: 'emoji.get_emoji_regexp()' is deprecated and will be removed in version 2.0.0. If you want to remove emoji from a string, consider the method emoji.replace_emoji(str, replace='').
To hide this warning, pin/downgrade the package to 'emoji~=1.6.3'
  em_split_emoji = emoji.get_emoji_regexp().split(em)


,tweet,class
0,ان الذين يعيشون على الارض ليسوا ملايكة بل بشر ...,pos
1,كل سنة وانتم طيبين,pos
2,و انتهى مشوار الخواجة,neg
3,مش عارف ابتدى مذاكره منين,neg
4,اختصروا الطريق بدلا من اختيار المنصف ثم الانقل...,neg
...,...,...
2054,الجمال مبيحتاح اي مكياج لناعم وله خشن جمل الطا...,neu
2055,نتمني وجود الفنانة رنا سماحة افضل فنانة صاعدة ...,neu
2056,ولد الهدى فالكاينات ضياء وفم الزمان تبسم وسناء...,pos
2057,انت متناقض جدا يا صلاح,neg


In [ ]:
!pip install farasapy

In [ ]:
!git clone https://github.com/aub-mind/arabert.git

In [ ]:
!pip install pyarabic

In [76]:
from arabert.preprocess import ArabertPreprocessor

model_name = "UBC-NLP/MARBERT"
arabert_prep = ArabertPreprocessor(model_name=model_name)

df['tweet']=df['tweet'].apply(lambda x: arabert_prep.preprocess(x))


# text = "ولن نبالغ إذا قلنا: إن 'هاتف' أو 'كمبيوتر المكتب' في زمننا هذا ضروري"
# arabert_prep.preprocess(text)
# # "و+ لن نبالغ إذا قل +نا : إن ' هاتف ' أو ' كمبيوتر ال+ مكتب ' في زمن +نا هذا ضروري"

FileNotFoundError: [WinError 2] The system cannot find the file specified

## Label Encoder

In [11]:
from sklearn import preprocessing
# Apply label encoding over the labels
lable_encoder = preprocessing.LabelEncoder()
encoded_labels =lable_encoder.fit_transform(df["class"])
df['class']=encoded_labels
df

,tweet,class
0,ان الذين يعيشون على الارض ليسوا ملايكة بل بشر ...,2
1,كل سنة وانتم طيبين,2
2,و انتهى مشوار الخواجة,0
3,مش عارف ابتدى مذاكره منين,0
4,اختصروا الطريق بدلا من اختيار المنصف ثم الانقل...,0
...,...,...
2054,الجمال مبيحتاح اي مكياج لناعم وله خشن جمل الطا...,1
2055,نتمني وجود الفنانة رنا سماحة افضل فنانة صاعدة ...,1
2056,ولد الهدى فالكاينات ضياء وفم الزمان تبسم وسناء...,2
2057,انت متناقض جدا يا صلاح,0


In [12]:
df['length']=df['tweet'].apply(lambda x:len(x.split(' ')))
df['length'].max()

34

In [1]:
import pandas as pd
df=pd.read_csv('preprocessed_train_large_bert.csv')
seed=0
from sklearn import preprocessing
# Apply label encoding over the labels
lable_encoder = preprocessing.LabelEncoder()
encoded_labels =lable_encoder.fit_transform(df["class"])
df['class']=encoded_labels
df

,Unnamed: 0,tweet,class
0,0,' # علمتني _ الحياه أن الذين يعيشون على الأرض ...,2
1,1,' # ميري _ كرسمس كل سنة وانتم طيبين [رابط] ',2
2,2,' و انتهى مشوار الخواجة ',0
3,3,' مش عارف ابتدى مذاكره منين : - ',0
4,4,' [مستخدم] إختصروا الطريق بدلا من إختيار المنص...,0
...,...,...,...
2054,2054,' [مستخدم] الجمال مبيحتاح اي مكياج لناعم وله خ...,1
2055,2055,' [مستخدم] نتمني وجود الفنانة رنا سماحة افضل ف...,1
2056,2056,' ولد الهدى فالكائنات ضياء . . وفم الزمان تبسم...,2
2057,2057,' [مستخدم] [مستخدم] انت متناقض جدا يا صلاح ',0


## Train Test Split

In [2]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation=train_test_split(df['tweet'], df['class'], test_size=0.2, random_state=seed)
X_validation

1826    ' هتفضل مشاكلي بيني و بين نفسي لحد ما اموت و ه...
1505       ' لو فشلنا ادينا حاولنا دا هيدي الامل لغيرنا '
1994    ' اذا جاء أجل الله لايقدم ولا يؤخر - الموت اقر...
1349               ' دايما بتاخد مننا اعز واغلى ماعندنا '
781     ' # لو _ سانتا _ كلوز _ مصري كآن هيلعن اليوم ا...
                              ...                        
596                                ' امراه رائعه [رابط] '
1887            ' مارلين مونرو حلوة فشخ ! . . ليه كدا ! '
942      ' # موطني _ هو تحت البطانيه و جنب شاحن الموبيل '
634     ' [مستخدم] [مستخدم] [مستخدم] [مستخدم] @ [رابط]...
19      ' [مستخدم] ها ايه رايك اقوله ايه بقا . . انا ب...
Name: tweet, Length: 412, dtype: object

## TF_IDF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
def tfidf_ngram(n_gram,X_train,X_val):
    vectorizer = TfidfVectorizer(ngram_range=(n_gram,n_gram))
    x_train_vec = vectorizer.fit_transform(X_train)
    x_test_vec = vectorizer.transform(X_val)
    return x_train_vec,x_test_vec

In [4]:
# Applying tfidf with 1-gram, 2-gram and 3-gram
tfidf_1g_transformation_train,tfidf_1g_transformation_validation= tfidf_ngram(1,X_train,X_validation)
tfidf_2g_transformation_train,tfidf_2g_transformation_validation= tfidf_ngram(2,X_train,X_validation)

## Machine learning models

In [5]:
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

models=[SVC(),XGBClassifier(),RandomForestClassifier(),DecisionTreeClassifier(),LogisticRegression()]
for m in models :
    m.fit(tfidf_2g_transformation_train,y_train)
    print(m.score(tfidf_2g_transformation_train,y_train))
    print(m.score(tfidf_2g_transformation_validation,y_validation))

0.994535519125683
0.3859223300970874
0.5343047965998786
0.3616504854368932
0.9963570127504554
0.42961165048543687
0.9963570127504554
0.41262135922330095
0.9914996964177292
0.39563106796116504


# Model and Tokenizer initialization

In [6]:
!pip install transformers

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

#============= Initialize Arabic Bert =============#
#load your pre_trained model with all its weights
# model_name= 'aubmindlab/bert-base-arabertv02'
model_name='UBC-NLP/MARBERT' #top
# model_name='asafaya/bert-base-arabic'
# model_name='AraBERTv0.2-Twitter-base'
# model_name='aubmindlab/bert-large-arabertv2'
# model_name='aubmindlab/bert-base-arabertv02-twitter'
# model_name='aubmindlab/bert-large-arabertv02-twitter'
# model_name='aubmindlab/aragpt2-base'

# model_name='aubmindlab/bert-base-arabertv2'
tokenizer =AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
# model=AutoModel.from_pretrained(model_name,output_hidden_states=True)

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

In [8]:
!git clone https://github.com/aub-mind/arabert.git

'git' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
!pip install pyarabic

In [10]:
# Tokenize the sentences using bert tokenizer
df["bert_tokens"] = df.tweet.apply(lambda x: tokenizer(x).tokens())
df["bert_tokens_ids"] = df.tweet.apply(lambda x: tokenizer(x).tokens())

In [11]:
df

,Unnamed: 0,tweet,class,bert_tokens,bert_tokens_ids
0,0,' # علمتني _ الحياه أن الذين يعيشون على الأرض ...,2,"[[CLS], ', [UNK], علمتني, [UNK], الحياه, ان, ا...","[[CLS], ', [UNK], علمتني, [UNK], الحياه, ان, ا..."
1,1,' # ميري _ كرسمس كل سنة وانتم طيبين [رابط] ',2,"[[CLS], ', [UNK], ميري, [UNK], كرس, ##مس, كل, ...","[[CLS], ', [UNK], ميري, [UNK], كرس, ##مس, كل, ..."
2,2,' و انتهى مشوار الخواجة ',0,"[[CLS], ', و, انتهى, مشوار, الخواجة, ', [SEP]]","[[CLS], ', و, انتهى, مشوار, الخواجة, ', [SEP]]"
3,3,' مش عارف ابتدى مذاكره منين : - ',0,"[[CLS], ', مش, عارف, ابتدى, مذاكره, منين, :, -...","[[CLS], ', مش, عارف, ابتدى, مذاكره, منين, :, -..."
4,4,' [مستخدم] إختصروا الطريق بدلا من إختيار المنص...,0,"[[CLS], ', [, مستخدم, ], اختصر, ##وا, الطريق, ...","[[CLS], ', [, مستخدم, ], اختصر, ##وا, الطريق, ..."
...,...,...,...,...,...
2054,2054,' [مستخدم] الجمال مبيحتاح اي مكياج لناعم وله خ...,1,"[[CLS], ', [, مستخدم, ], الجمال, مبيح, ##تاح, ...","[[CLS], ', [, مستخدم, ], الجمال, مبيح, ##تاح, ..."
2055,2055,' [مستخدم] نتمني وجود الفنانة رنا سماحة افضل ف...,1,"[[CLS], ', [, مستخدم, ], نتمني, وجود, الفنانة,...","[[CLS], ', [, مستخدم, ], نتمني, وجود, الفنانة,..."
2056,2056,' ولد الهدى فالكائنات ضياء . . وفم الزمان تبسم...,2,"[[CLS], ', ولد, الهدى, فالك, ##اينات, ضياء, .,...","[[CLS], ', ولد, الهدى, فالك, ##اينات, ضياء, .,..."
2057,2057,' [مستخدم] [مستخدم] انت متناقض جدا يا صلاح ',0,"[[CLS], ', [, مستخدم, ], [, مستخدم, ], انت, مت...","[[CLS], ', [, مستخدم, ], [, مستخدم, ], انت, مت..."


In [12]:
df["encoded"] = df.tweet.apply(lambda x: tokenizer.encode_plus(x,return_tensors='pt')['input_ids'])

In [13]:
df

,Unnamed: 0,tweet,class,bert_tokens,bert_tokens_ids,encoded
0,0,' # علمتني _ الحياه أن الذين يعيشون على الأرض ...,2,"[[CLS], ', [UNK], علمتني, [UNK], الحياه, ان, ا...","[[CLS], ', [UNK], علمتني, [UNK], الحياه, ان, ا...","[[tensor(2), tensor(9), tensor(1), tensor(8244..."
1,1,' # ميري _ كرسمس كل سنة وانتم طيبين [رابط] ',2,"[[CLS], ', [UNK], ميري, [UNK], كرس, ##مس, كل, ...","[[CLS], ', [UNK], ميري, [UNK], كرس, ##مس, كل, ...","[[tensor(2), tensor(9), tensor(1), tensor(5390..."
2,2,' و انتهى مشوار الخواجة ',0,"[[CLS], ', و, انتهى, مشوار, الخواجة, ', [SEP]]","[[CLS], ', و, انتهى, مشوار, الخواجة, ', [SEP]]","[[tensor(2), tensor(9), tensor(144), tensor(76..."
3,3,' مش عارف ابتدى مذاكره منين : - ',0,"[[CLS], ', مش, عارف, ابتدى, مذاكره, منين, :, -...","[[CLS], ', مش, عارف, ابتدى, مذاكره, منين, :, -...","[[tensor(2), tensor(9), tensor(2093), tensor(3..."
4,4,' [مستخدم] إختصروا الطريق بدلا من إختيار المنص...,0,"[[CLS], ', [, مستخدم, ], اختصر, ##وا, الطريق, ...","[[CLS], ', [, مستخدم, ], اختصر, ##وا, الطريق, ...","[[tensor(2), tensor(9), tensor(35), tensor(789..."
...,...,...,...,...,...,...
2054,2054,' [مستخدم] الجمال مبيحتاح اي مكياج لناعم وله خ...,1,"[[CLS], ', [, مستخدم, ], الجمال, مبيح, ##تاح, ...","[[CLS], ', [, مستخدم, ], الجمال, مبيح, ##تاح, ...","[[tensor(2), tensor(9), tensor(35), tensor(789..."
2055,2055,' [مستخدم] نتمني وجود الفنانة رنا سماحة افضل ف...,1,"[[CLS], ', [, مستخدم, ], نتمني, وجود, الفنانة,...","[[CLS], ', [, مستخدم, ], نتمني, وجود, الفنانة,...","[[tensor(2), tensor(9), tensor(35), tensor(789..."
2056,2056,' ولد الهدى فالكائنات ضياء . . وفم الزمان تبسم...,2,"[[CLS], ', ولد, الهدى, فالك, ##اينات, ضياء, .,...","[[CLS], ', ولد, الهدى, فالك, ##اينات, ضياء, .,...","[[tensor(2), tensor(9), tensor(3735), tensor(4..."
2057,2057,' [مستخدم] [مستخدم] انت متناقض جدا يا صلاح ',0,"[[CLS], ', [, مستخدم, ], [, مستخدم, ], انت, مت...","[[CLS], ', [, مستخدم, ], [, مستخدم, ], انت, مت...","[[tensor(2), tensor(9), tensor(35), tensor(789..."


## Padding and attention mask

In [14]:
from keras_preprocessing.sequence import pad_sequences

# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway.
# In the original paper, the authors used a length of 512.
MAX_LEN = 64
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in df['bert_tokens']]
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

In [15]:

import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

# Use train_test_split to split our data into train and validation sets for training
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, encoded_labels,
                                                            random_state=seed, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=seed, test_size=0.1)
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 64

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop,
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(train_data, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(validation_data, batch_size=batch_size)

## Set optimizer parameters

In [16]:
import torch.optim as optim

param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],'weight_decay_rate': 0.01},
                                {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],'weight_decay_rate': 0.0}]
# This variable contains all of the hyperparemeter information our training loop needs
# optimizer = optim.BertAdam(optimizer_grouped_parameters,lr=2e-5,warmup=.1)
optimizer = optim.AdamW(optimizer_grouped_parameters,lr=5e-6)

# Training

In [17]:
from tqdm import trange
import numpy as np
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)
t = []

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs
epochs = 11

# Transfer the model to GPU
model.to("cuda")

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):


  # Training

  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    b_input_ids, b_input_mask, b_labels = batch
    b_labels = b_labels.type(torch.LongTensor)   # casting to long
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()

    # Forward pass
    loss = model(b_input_ids.to("cuda"), token_type_ids=None, attention_mask=b_input_mask.to("cuda"), labels=b_labels.to("cuda"))["loss"]
    train_loss_set.append(loss.item())

    # Backward pass
    loss.backward()

    # Update parameters and take a step using the computed gradient
    optimizer.step()


    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))

  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    # batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    b_labels = b_labels.type(torch.LongTensor)   # casting to long
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids.to("cuda"), token_type_ids=None, attention_mask=b_input_mask.to("cuda"))

    # Move logits and labels to CPU
    logits = logits["logits"].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/11 [00:00<?, ?it/s]

Train loss: 1.0659035279833038


Epoch:   9%|▉         | 1/11 [00:12<02:09, 12.98s/it]

Validation Accuracy: 0.5887276785714286
Train loss: 0.8915698384416515


Epoch:  18%|█▊        | 2/11 [00:24<01:46, 11.86s/it]

...

Validation Accuracy: 0.7589285714285714
Train loss: 0.37091002587614386


Epoch:  45%|████▌     | 5/11 [00:57<01:08, 11.35s/it]

Validation Accuracy: 0.7511160714285714


Epoch:  45%|████▌     | 5/11 [00:59<01:11, 11.92s/it]


KeyboardInterrupt: 

In [18]:
#============= Read CSV and apply data preperation =============#
df_submit = pd.read_csv("test.csv")

# # clean-up: remove #tags, http links and special symbols
# df_submit.tweet = df_submit.tweet.apply(lambda x: x[2:-2])
# df_submit.tweet = df_submit.tweet.apply(lambda x: re.sub(r'http\S+', '', x))
# df_submit.tweet = df_submit.tweet.apply(lambda x: re.sub(r'[@|#]\S*', '', x))
# df_submit.tweet = df_submit.tweet.apply(lambda x: re.sub(r'"+', '', x))
#
# # Remove arabic signs
# df_submit.tweet = df_submit.tweet.apply(lambda x: re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', x))
#
# # Remove repeated letters like "الللللللللللللللله" to "الله"
# df_submit.tweet = df_submit.tweet.apply(lambda x: x[0:2] + ''.join([x[i] for i in range(2, len(x)) if x[i]!=x[i-1] or x[i]!=x[i-2]]))
#
# # remove stop words
# df_submit.iloc[:,0] = df_submit.iloc[:,0].apply(lambda x: '' if x in arabic_stop_words else x)


# df_submit=data_preprocessing(df_submit)
#
# df_submit["tweet"] = df_submit.tweet.apply(lambda x: data_cleaning(x))

df_submit=pd.read_csv('preprocessed_test_UBC.csv')

# Tokenize the sentences using bert tokenizer
df_submit["bert_tokens"] = df_submit.tweet.apply(lambda x: tokenizer(x).tokens())

In [19]:
bert_tokens_submit = df_submit["bert_tokens"]

In [20]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 64
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids_submit = [tokenizer.convert_tokens_to_ids(x) for x in bert_tokens_submit]
# Pad our input tokens
input_ids_submit = pad_sequences(input_ids_submit, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
# Create attention masks
attention_masks_submit = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids_submit:
  seq_mask = [float(i>0) for i in seq]
  attention_masks_submit.append(seq_mask)

In [21]:
# Convert all of our data into torch tensors, the required datatype for our model
inputs_submit = torch.tensor(input_ids_submit)
masks_submit = torch.tensor(attention_masks_submit)

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
batch_size = 64
submit_data = TensorDataset(inputs_submit, masks_submit)

# do not use shuffle, we need the preds to be in same order
submit_dataloader = DataLoader(submit_data, batch_size=batch_size)#, shuffle=True)

In [22]:
# Put the model in an evaluation state
model.eval()

# Transfer model to GPU
model.to("cuda")

outputs = []
for input, masks in submit_dataloader:
  torch.cuda.empty_cache() # empty the gpu memory

  # Transfer the batch to gpu
  input = input.to('cuda')
  masks = masks.to('cuda')

  # Run inference on the batch
  output = model(input, attention_mask=masks)["logits"]

  # Transfer the output to CPU again and convert to numpy
  output = output.cpu().detach().numpy()

  # Store the output in a list
  outputs.append(output)

# Concatenate all the lists within the list into one list
outputs = [x for y in outputs for x in y]

# Inverse transform the label encoding
pred_flat = np.argmax(outputs, axis=1).flatten()
output_labels = lable_encoder.inverse_transform(pred_flat)

In [23]:
submission = pd.DataFrame({"Id":np.arange(1, len(output_labels)+1), "class":output_labels})
# save (submission)
submission.to_csv("submission23.csv", index=False)